In [ ]:
import polars as pl

### Expressions for defining columns

In [18]:
def payer_category():
    return (
        pl.when(pl.col('payer_name') == 'Gross_Charge').then('gross')
        .when(pl.col('payer_name') == 'Cash_Charge').then('cash')
        .when(pl.col('payer_name') == 'Min_Negotiated_Rate').then('min')
        .when(pl.col('payer_name') == 'Max_Negotiated_Rate').then('max')
        .otherwise('payer').alias('payer_category')
    )

def hcpcs_cpt_and_drg():
    
    hcpcs_cpt = pl.when(pl.col('line_type') == '2-CPT').then(pl.col('code')).alias('hcpcs_cpt')
    ms_drg = pl.when(pl.col('line_type') == '3-DRG').then(pl.col('code')).str.zfill(3).alias('ms_drg')
    
    return hcpcs_cpt, ms_drg

def all_charges():
    
    standard_charge_percent = (
        pl.col('standard_charge')
        .str.extract('(\d+)% of gross charges')
        .alias('standard_charge_percent')
    )
    
    contracting_method = (
        pl.when(pl.col('standard_charge').str.contains('\% of gross charges'))
        .then('percent of total billed charge')
        .alias('contracting_method')
    )
    
    standard_charge = (
        pl.when(pl.col('standard_charge').str.contains('\% of gross charges'))
        .then(None).otherwise(pl.col('standard_charge')).alias('standard_charge')
    )

    return standard_charge, standard_charge_percent, contracting_method

### The following code transforms the entire file in one pass

In [23]:
file = '474063406_Ascension Saint Thomas Highlands_standardcharges.xlsx'

read_csv_options = {
    'skip_rows': 1, 
    'infer_schema_length': 0, 
    'null_values': ['N/A']
}

In [22]:
(
    pl.read_excel(file, sheet_id = 2, read_csv_options = read_csv_options)
    .slice(3,)
    .drop(['Facility_BU_ID', 'UB_Revenue_Description'])
    .rename({
        'Code_Type':'line_type',
        'Code':'code',
        'Description':'description',
        'UB_Revenue_Code':'rev_code',
    })
    .melt(
        ['line_type', 'code', 'description', 'rev_code'],
        variable_name = 'payer_name',
        value_name = 'standard_charge',
    )
    .filter(pl.col('standard_charge').is_not_null())
    .with_columns([
        payer_category(),
        pl.col('rev_code').str.zfill(4).keep_name(),
        *hcpcs_cpt_and_drg(),
        *all_charges(),
    ])
)

line_type,code,description,rev_code,payer_name,standard_charge,payer_category,hcpcs_cpt,ms_drg,standard_charge_percent,contracting_method
str,str,str,str,str,str,str,str,str,str,str
"""1-CDM""","""1600044""","""BUPIVACAINE/EP…","""0251""","""Gross_Charge""","""33.65""","""gross""",null,null,null,null
"""1-CDM""","""1600206""","""GENTAMICIN 40 …","""0636""","""Gross_Charge""","""23""","""gross""",null,null,null,null
"""1-CDM""","""1600273""","""LIDOCAINE/EPI …","""0250""","""Gross_Charge""","""24.8""","""gross""",null,null,null,null
"""1-CDM""","""1600486""","""TOBRAMYCIN 40 …","""0636""","""Gross_Charge""","""23.65""","""gross""",null,null,null,null
"""1-CDM""","""1600648""","""AMP-SULBAC 1.5…","""0636""","""Gross_Charge""","""36.5""","""gross""",null,null,null,null
"""1-CDM""","""1600680""","""VANCOMYCIN 500…","""0636""","""Gross_Charge""","""55.95""","""gross""",null,null,null,null
"""1-CDM""","""1600788""","""METRONIDAZOLE …","""0251""","""Gross_Charge""","""15.45""","""gross""",null,null,null,null
"""1-CDM""","""1600923""","""D5% 100ML""","""0258""","""Gross_Charge""","""154.35""","""gross""",null,null,null,null
"""1-CDM""","""1600931""","""NACL 0.9% 100 …","""0258""","""Gross_Charge""","""154.35""","""gross""",null,null,null,null
